In [1]:
import sys
sys.path.append('..')
from db1 import *

R = PolynomialRing(QQ, 't')
S = PolynomialRing(ZZ, 't')
t = R.gen()

def is_paving(M):
    n = M.size()
    r = M.rank()
    return (len(M.independent_r_sets(r-1)) == binomial(n, r-1))

def q_kl(k, h):
    return kazhdan_lusztig_inverse_uniform(k, h+1) - kazhdan_lusztig_inverse_uniform(k-1, h)

def kl_inverse_fast(M):
    if M.loops(): return R(0)
    k, n = M.rank(), M.size()
    if k == n or k == 0: return R(1)
    if not M.is_connected():
        ans = R(1)
        CC = M.components()
        for N in CC:
            res = M.delete(M.groundset() - N)
            ans = ans * kl_inverse_fast(res)
        return ans

    if is_paving(M):
        return kl_inverse_paving(M)
    if is_paving(M.dual()):
        return kl_inverse_copaving(M)
    """
    if n <= 8 and M.is_connected():
        for i in range(len(mat[n][k])):
            if mat[n][k][i].is_isomorphic(M):
                return ikl[n][k][i]
    """
    LF = M.lattice_of_flats()
    ans = R(0)
    for F in LF:
        if len(F) != n:
            Res = M.delete(M.groundset() - F)
            Con = M.contract(F)
            chi = characteristic_polynomial(Con)(1/t) * t**(Con.rank())
            PPP = kl_inverse_fast(Res)(t) * (-1)**(Res.rank())
            ans = ans + chi * PPP
    assert (t**k * ans(1/t)).numerator() == -ans(t)
    ans = ans.numerator() * (-1)**(k+1)
    return ans.truncate((k+1)//2)

def kazhdan_lusztig_inverse_uniform(k, n):
    if k == n:
        return R(1)
    d = k
    m = n - d
    ans = 0
    for j in range((d-1)//2 + 1):
        ans = ans + m * (d-2*j)/((m+j) * (m+d-j)) * binomial(d, j) * t**j
    return ans * binomial(m+d, d)

def kl_inverse_paving(M):
    assert is_paving(M)
    n = M.size()
    k = M.rank()
    ans = kazhdan_lusztig_inverse_uniform(k, n)
    for H in M.hyperplanes():
        h = len(H)
        if h >= k:
            ans = ans - q_kl(k, h)
    return ans

def kl_inverse_copaving(M):
    assert is_paving(M.dual())
    n = M.size()
    k = M.rank()
    ans = kazhdan_lusztig_inverse_uniform(k, n)
    for H in M.dual().hyperplanes():
        h = len(H)
        if h >= n-k:
            ans = ans - kli_vtilde_dual(n-k, h, n) + kazhdan_lusztig_inverse_uniform(h-n+k+1, h) * kazhdan_lusztig_inverse_uniform(n-h-1, n-h)
    return ans

def kli_vtilde_dual(k, h, n):
    return helper1(n-k, h, n)

def helper1(k, h, n):
    c = n - h
    ans1 = kazhdan_lusztig_inverse_uniform(k, n)
    ans2 = helper2(c, k, n)
    ans3 = kazhdan_lusztig_inverse_uniform(k-c+1, h) * kazhdan_lusztig_inverse_uniform(c-1, c)
    return ans1 - ans2 + ans3

def helper2(c, k, n):
    h = n - c
    ans = 0
    for j in range(k-c+1):
        ans = ans + binomial(n-c, j) * (-1)**(c-1+j) * kazhdan_lusztig_inverse_uniform(c-1, c) * t**(k-c-j+1) * chuly(k-c-j+1, n-c-j)(1/t)
    for i in range(c-1):
        for j in range(k-i):
            ans = ans + binomial(c, i) * binomial(n-c, j) * (-1)**(i+j) * t**(k-i-j) * helper4(c, k, n, i, j)(1/t)
    ans = ans.numerator().truncate((k-1)//2 + 1)
    if ans[0] < 0:
        ans = -ans
    return ans

def helper3(c, k, n):
    ans = 0
    for j in range(k-c+1):
        ans = ans + binomial(n-c, j) * kazhdan_lusztig_uniform_matroid(c-1, c) * (-1)**(k-c-j+1) * kazhdan_lusztig_inverse_uniform(k-c-j+1, n-c-j)
    for i in range(c-1):
        for j in range(k-i):
            ans = ans + binomial(c, i) * binomial(n-c, j) * (-1)**(k-i-j) * helper2(c-i, k-i-j, n-i-j)
    return -ans

def helper4(c, k, n, i, j):
    ans = 0
    for l in range(c-i-1):
        ans = ans + (-1)**l * (t-1)**(max(n-i-j-l-1, 0))
    for u in range(n-k-1):
        ans = doit_once(ans)
    return ans

def chuly(a, b):
    ans = (t-1)**b
    for i in range(b-a):
        ans = doit_once(ans)
    return ans

def doit_once(p):
    p = p // t**2
    p = p * t
    p = p - p(1)
    return p

def lorenzo(k, h, n):
    c = n - h
    ans1 = kazhdan_lusztig_uniform_matroid(k, n) + kazhdan_lusztig_uniform_matroid(k-c+1, h) * kazhdan_lusztig_uniform_matroid(c-1, c)
    ans2 = helper3(c, k, n)
    return ans1 - ans2

In [2]:
def qhat(M):
    return (-1)^M.rank() * kl_inverse_fast(M)

def the_set_S(flats, e):
    return set(F for F in flats if set(e).issubset(set(F)) and set(F) - set(e) not in flats)

def extra_sum(M, S, e):
    return sum(qhat(M.contract(F)) * lt(M.minor(contractions=e, deletions=M.groundset() - F)) for F in S)

def parallel_connection(m, n):
    H = graphs.CycleGraph(m + n - 2)
    G = Graph(H, loops = true, multiedges = true)
    G.add_edges({(0, m-1)})
    edge_e = frozenset({(0, m - 1)})
    return G

def leading_term(p):
    return p.leading_coefficient() * t ** p.degree()

def lt(M):
    r = M.rank()
    cloops = M.coloops()
    if len(cloops) > 0 or r % 2 == 0:
        return 0
    else:
        return leading_term(qhat(M))

matroids_list = [matroids.Uniform(r, n) for r, n in [(2, 3), (2, 4), (2,5), (3, 4), (4, 5), (5,6), (3, 5), (4, 6)]] + \
    [matroids.Wheel(i) for i in range(3, 6)] + \
    [matroids.Whirl(i) for i in range(3, 5)] + \
    [Matroid(parallel_connection(n, n)) for n in range(3, 7)]

es = [next({e} for e in m.groundset() if e not in m.coloops()) for m in matroids_list]

In [3]:
import pandas as pd

qhat = kl_inverse_fast

data = {
    'Matroid': matroids_list,
    'qhat': [],
    'qhatDel': [],
    '(t+1)*qhatCon': [],
    't*extra-sum': [],
    'rhs': [],
    'lhs - rhs': []
}

for (m, e) in zip(matroids_list, es):
    flats = list(m.lattice_of_flats())
    data['qhat'].append(qhat(m))
    data['qhatDel'].append(qhat(m.delete(e)))
    data['(t+1)*qhatCon'].append((t+1) * qhat(m.contract(e)))
    data['t*extra-sum'].append(t * extra_sum(m, the_set_S(flats, e), e))
    data['rhs'].append(data['qhatDel'][-1] - data['(t+1)*qhatCon'][-1] + \
        data['t*extra-sum'][-1])
    data['lhs - rhs'].append(data['qhat'][-1] - data['rhs'][-1])

df = pd.DataFrame(data)
display(df)

,Matroid,qhat,qhatDel,(t+1)*qhatCon,t*extra-sum,rhs,lhs - rhs
0,"U(2, 3): Matroid of rank 2 on 3 elements with ...",2,1,t + 1,t,0,2
1,"U(2, 4): Matroid of rank 2 on 4 elements with ...",3,2,t + 1,t,1,2
2,"U(2, 5): Matroid of rank 2 on 5 elements with ...",4,3,t + 1,t,2,2
3,"U(3, 4): Matroid of rank 3 on 4 elements with ...",2*t + 3,1,2*t + 2,0,-2*t - 1,4*t + 4
4,"U(4, 5): Matroid of rank 4 on 5 elements with ...",5*t + 4,1,2*t^2 + 5*t + 3,2*t^2,-5*t - 2,10*t + 6
5,"U(5, 6): Matroid of rank 5 on 6 elements with ...",5*t^2 + 9*t + 5,1,5*t^2 + 9*t + 4,0,-5*t^2 - 9*t - 3,10*t^2 + 18*t + 8
6,"U(3, 5): Matroid of rank 3 on 5 elements with ...",5*t + 6,2*t + 3,3*t + 3,0,-t,6*t + 6
7,"U(4, 6): Matroid of rank 4 on 6 elements with ...",16*t + 10,5*t + 4,5*t^2 + 11*t + 6,5*t^2,-6*t - 2,22*t + 12
8,Wheel(3): Regular matroid of rank 3 on 6 eleme...,t + 6,t + 4,2*t + 2,2*t,t + 2,4
9,Wheel(4): Regular matroid of rank 4 on 8 eleme...,9*t + 14,8*t + 10,t^2 + 5*t + 4,t^2 + 4*t,7*t + 6,2*t + 8


In [4]:
data = {
    'Matroid': matroids_list,
    'qhat': [],
    'qhatDel': [],
    '(t+1)*qhatCon': [],
    't*extra-sum': [],
    'lhs - rhs': []
}

for (m, e) in zip(matroids_list, es):
    flats = list(m.lattice_of_flats())
    data['qhat'].append(qhat(m))
    data['qhatDel'].append(qhat(m.delete(e)))
    data['(t+1)*qhatCon'].append((t+1) * qhat(m.contract(e)))
    data['t*extra-sum'].append(t * extra_sum(m, the_set_S(flats, e), e))
    data['lhs - rhs'].append(data['qhat'][-1] - \
        data['qhatDel'][-1] + data['(t+1)*qhatCon'][-1] - \
        data['t*extra-sum'][-1])

df = pd.DataFrame(data)
display(df)

,Matroid,qhat,qhatDel,(t+1)*qhatCon,t*extra-sum,lhs - rhs
0,"U(2, 3): Matroid of rank 2 on 3 elements with ...",2,1,t + 1,t,2
1,"U(2, 4): Matroid of rank 2 on 4 elements with ...",3,2,t + 1,t,2
2,"U(2, 5): Matroid of rank 2 on 5 elements with ...",4,3,t + 1,t,2
3,"U(3, 4): Matroid of rank 3 on 4 elements with ...",2*t + 3,1,2*t + 2,0,4*t + 4
4,"U(4, 5): Matroid of rank 4 on 5 elements with ...",5*t + 4,1,2*t^2 + 5*t + 3,2*t^2,10*t + 6
5,"U(5, 6): Matroid of rank 5 on 6 elements with ...",5*t^2 + 9*t + 5,1,5*t^2 + 9*t + 4,0,10*t^2 + 18*t + 8
6,"U(3, 5): Matroid of rank 3 on 5 elements with ...",5*t + 6,2*t + 3,3*t + 3,0,6*t + 6
7,"U(4, 6): Matroid of rank 4 on 6 elements with ...",16*t + 10,5*t + 4,5*t^2 + 11*t + 6,5*t^2,22*t + 12
8,Wheel(3): Regular matroid of rank 3 on 6 eleme...,t + 6,t + 4,2*t + 2,2*t,4
9,Wheel(4): Regular matroid of rank 4 on 8 eleme...,9*t + 14,8*t + 10,t^2 + 5*t + 4,t^2 + 4*t,2*t + 8
